In [1]:
import pandas as pd
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import pickle
from tools import load_data, model_fit

In [2]:
X, y, site = load_data()

2022-07-06 01:39:20.705335: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from sklearn.utils import shuffle, resample

In [4]:
from sklearn.metrics import r2_score, median_absolute_error

In [5]:
model_dict = {
  "cnn_lenet": {"model": cnn_lenet, "lr": 0.001}, 
  # "mlp4": {"model": mlp4, "lr": 0.001},
  # "cnn_vgg": {"model": cnn_vgg, "lr": 0.001},
  # "lstm1v0": {"model": lstm1v0, "lr": 0.01},
  # "lstm1": {"model": lstm1, "lr": 0.01},
  # "lstm2": {"model": lstm2, "lr": 0.01},
  # "blstm1": {"model": blstm1, "lr": 0.01},
  # "blstm2": {"model": blstm1, "lr": 0.01},
  # "lstm_fcn": {"model": lstm_fcn, "lr": 0.01},
#   "cnn_resnet": {"model": cnn_resnet, "lr": 0.01}
             }

In [6]:
n_runs = 10

In [7]:
# Generated once with this code and then hard-coded:
# seeds = np.array([np.abs(np.floor(np.random.randn()*1000)) for ii in range(n_runs)], dtype=int)

seeds = np.array([484, 645, 714, 244, 215, 1502, 1334, 1576, 469, 1795])

In [8]:
def fit_and_eval(model, X, y, random_state, batch_size=128, n_epochs=1000, augment=None, train_size=None):
    model_func = model_dict[model]["model"]
    lr = model_dict[model]["lr"]
    X_train, X_test, y_train, y_test, site_train, site_test = train_test_split(X, y, site, test_size=0.2, random_state=random_state)
    imputer = SimpleImputer(strategy="median")
    # If train_size is set, select train_size subjects to be the training data:
    if train_size is not None:
        X_train, y_train, site_train = resample(X_train, y_train, site_train, replace=False, n_samples=train_size, random_state=random_state)
    
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([imputer.fit_transform(X_test[..., ii])[:, :, None] for ii in range(X_test.shape[-1])], -1)
    # Combat
    X_train = np.concatenate([CombatModel().fit_transform(X_train[..., ii], site_train[:, None], None, None)[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([CombatModel().fit_transform(X_test[..., ii], site_test[:, None], None, None)[:, :, None] for ii in range(X_test.shape[-1])], -1)
    
    trained = model_fit(model_func, X_train, y_train, lr, 
                        batch_size=batch_size, n_epochs=n_epochs)
    metric = []
    value = []
    
    y_pred = trained.predict(X_test)
    metric.append("mae")
    value.append(mean_absolute_error(y_test, y_pred))
    metric.append("mad")
    value.append(median_absolute_error(y_test, y_pred))
    metric.append("r2")
    value.append(r2_score(y_test, y_pred))
    
    result = {'Model': [model] * len(metric),
              'Metric': metric,
              'Value': value}
    
    return pd.DataFrame(result), pd.DataFrame(dict(y_pred=y_pred.squeeze(), y_test=y_test))

In [9]:
dfs_eval = []
dfs_pred = []

In [10]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=175)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("lenet_5_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("lenet_5_pred.csv")

##################################################
model:  cnn_lenet
run:  0
pooling layers: 4
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 conv1d (Conv1D)             (None, 100, 6)            1302      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 50, 6)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 16)            304       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 25, 16)           0         
 1D)                                                             
                                

Epoch 21/1000
2/2 [==============================] - 0s 19ms/step - loss: 21.3055 - mean_squared_error: 21.3055 - rmse: 4.6158 - mean_absolute_error: 3.6669 - val_loss: 8.2581 - val_mean_squared_error: 8.2581 - val_rmse: 2.8737 - val_mean_absolute_error: 2.3380 - lr: 0.0010
Epoch 22/1000
2/2 [==============================] - 0s 19ms/step - loss: 25.7660 - mean_squared_error: 25.7660 - rmse: 5.0760 - mean_absolute_error: 4.1132 - val_loss: 7.1874 - val_mean_squared_error: 7.1874 - val_rmse: 2.6809 - val_mean_absolute_error: 2.1958 - lr: 0.0010
Epoch 23/1000
2/2 [==============================] - 0s 24ms/step - loss: 19.1252 - mean_squared_error: 19.1252 - rmse: 4.3732 - mean_absolute_error: 3.3099 - val_loss: 6.6051 - val_mean_squared_error: 6.6051 - val_rmse: 2.5700 - val_mean_absolute_error: 2.0999 - lr: 0.0010
Epoch 24/1000
1/2 [==============>...............] - ETA: 0s - loss: 20.6412 - mean_squared_error: 20.6412 - rmse: 4.5433 - mean_absolute_error: 3.7706
Epoch 00024: ReduceLROn

Epoch 49/1000
2/2 [==============================] - 0s 24ms/step - loss: 22.0123 - mean_squared_error: 22.0123 - rmse: 4.6917 - mean_absolute_error: 3.6256 - val_loss: 7.4105 - val_mean_squared_error: 7.4105 - val_rmse: 2.7222 - val_mean_absolute_error: 2.2331 - lr: 2.5000e-04
Epoch 50/1000
2/2 [==============================] - 0s 25ms/step - loss: 19.4672 - mean_squared_error: 19.4672 - rmse: 4.4122 - mean_absolute_error: 3.3545 - val_loss: 7.5615 - val_mean_squared_error: 7.5615 - val_rmse: 2.7498 - val_mean_absolute_error: 2.2548 - lr: 2.5000e-04
Epoch 51/1000
2/2 [==============================] - 0s 24ms/step - loss: 18.3668 - mean_squared_error: 18.3668 - rmse: 4.2856 - mean_absolute_error: 3.4188 - val_loss: 7.7509 - val_mean_squared_error: 7.7509 - val_rmse: 2.7840 - val_mean_absolute_error: 2.2804 - lr: 2.5000e-04
Epoch 52/1000
2/2 [==============================] - 0s 23ms/step - loss: 18.7130 - mean_squared_error: 18.7130 - rmse: 4.3258 - mean_absolute_error: 3.4021 - val_

Epoch 78/1000
2/2 [==============================] - 0s 27ms/step - loss: 19.5132 - mean_squared_error: 19.5132 - rmse: 4.4174 - mean_absolute_error: 3.5841 - val_loss: 7.4362 - val_mean_squared_error: 7.4362 - val_rmse: 2.7269 - val_mean_absolute_error: 2.2411 - lr: 1.2500e-04
Epoch 79/1000
2/2 [==============================] - 0s 24ms/step - loss: 22.4617 - mean_squared_error: 22.4617 - rmse: 4.7394 - mean_absolute_error: 3.7061 - val_loss: 7.5230 - val_mean_squared_error: 7.5230 - val_rmse: 2.7428 - val_mean_absolute_error: 2.2535 - lr: 1.2500e-04
Epoch 80/1000
2/2 [==============================] - 0s 245ms/step - loss: 19.4998 - mean_squared_error: 19.4998 - rmse: 4.4159 - mean_absolute_error: 3.4919 - val_loss: 7.5942 - val_mean_squared_error: 7.5942 - val_rmse: 2.7558 - val_mean_absolute_error: 2.2635 - lr: 1.2500e-04
Epoch 81/1000
2/2 [==============================] - 0s 35ms/step - loss: 16.2712 - mean_squared_error: 16.2712 - rmse: 4.0338 - mean_absolute_error: 3.2469 - val

2/2 [==============================] - 2s 306ms/step - loss: 159.1763 - mean_squared_error: 159.1763 - rmse: 12.6165 - mean_absolute_error: 12.0947 - val_loss: 103.6571 - val_mean_squared_error: 103.6571 - val_rmse: 10.1812 - val_mean_absolute_error: 9.6292 - lr: 0.0010
Epoch 2/1000
2/2 [==============================] - 0s 53ms/step - loss: 111.7433 - mean_squared_error: 111.7433 - rmse: 10.5709 - mean_absolute_error: 10.0385 - val_loss: 75.9148 - val_mean_squared_error: 75.9148 - val_rmse: 8.7129 - val_mean_absolute_error: 8.0887 - lr: 0.0010
Epoch 3/1000
2/2 [==============================] - 0s 52ms/step - loss: 88.1579 - mean_squared_error: 88.1579 - rmse: 9.3892 - mean_absolute_error: 8.7585 - val_loss: 44.8185 - val_mean_squared_error: 44.8185 - val_rmse: 6.6947 - val_mean_absolute_error: 5.9086 - lr: 0.0010
Epoch 4/1000
2/2 [==============================] - 0s 44ms/step - loss: 54.9626 - mean_squared_error: 54.9626 - rmse: 7.4137 - mean_absolute_error: 6.5510 - val_loss: 16.61

...

Epoch 99/1000
2/2 [==============================] - 0s 22ms/step - loss: 16.2407 - mean_squared_error: 16.2407 - rmse: 4.0300 - mean_absolute_error: 3.2679 - val_loss: 14.4613 - val_mean_squared_error: 14.4613 - val_rmse: 3.8028 - val_mean_absolute_error: 2.9348 - lr: 6.2500e-05
Epoch 100/1000
2/2 [==============================] - 0s 19ms/step - loss: 12.7010 - mean_squared_error: 12.7010 - rmse: 3.5638 - mean_absolute_error: 2.7285 - val_loss: 14.4390 - val_mean_squared_error: 14.4390 - val_rmse: 3.7999 - val_mean_absolute_error: 2.9325 - lr: 6.2500e-05
Epoch 101/1000
2/2 [==============================] - 0s 20ms/step - loss: 15.8183 - mean_squared_error: 15.8183 - rmse: 3.9772 - mean_absolute_error: 3.1541 - val_loss: 14.4274 - val_mean_squared_error: 14.4274 - val_rmse: 3.7983 - val_mean_absolute_error: 2.9313 - lr: 6.2500e-05
Epoch 102/1000
2/2 [==============================] - 0s 23ms/step - loss: 17.3094 - mean_squared_error: 17.3094 - rmse: 4.1605 - mean_absolute_error: 3.31

2/2 [==============================] - 2s 367ms/step - loss: 169.4136 - mean_squared_error: 169.4136 - rmse: 13.0159 - mean_absolute_error: 12.2494 - val_loss: 114.0272 - val_mean_squared_error: 114.0272 - val_rmse: 10.6784 - val_mean_absolute_error: 9.8981 - lr: 0.0010
Epoch 2/1000
2/2 [==============================] - 0s 46ms/step - loss: 116.7851 - mean_squared_error: 116.7851 - rmse: 10.8067 - mean_absolute_error: 9.9898 - val_loss: 71.9120 - val_mean_squared_error: 71.9120 - val_rmse: 8.4801 - val_mean_absolute_error: 7.5277 - lr: 0.0010
Epoch 3/1000
2/2 [==============================] - 0s 44ms/step - loss: 76.0387 - mean_squared_error: 76.0387 - rmse: 8.7200 - mean_absolute_error: 7.7645 - val_loss: 37.8008 - val_mean_squared_error: 37.8008 - val_rmse: 6.1482 - val_mean_absolute_error: 4.9247 - lr: 0.0010
Epoch 4/1000
2/2 [==============================] - 0s 42ms/step - loss: 47.7313 - mean_squared_error: 47.7313 - rmse: 6.9088 - mean_absolute_error: 5.5305 - val_loss: 15.347

Epoch 30/1000
2/2 [==============================] - 0s 21ms/step - loss: 24.8512 - mean_squared_error: 24.8512 - rmse: 4.9851 - mean_absolute_error: 3.8040 - val_loss: 14.3480 - val_mean_squared_error: 14.3480 - val_rmse: 3.7879 - val_mean_absolute_error: 3.0397 - lr: 5.0000e-04
Epoch 31/1000
2/2 [==============================] - 0s 19ms/step - loss: 19.4979 - mean_squared_error: 19.4979 - rmse: 4.4156 - mean_absolute_error: 3.5021 - val_loss: 14.4014 - val_mean_squared_error: 14.4014 - val_rmse: 3.7949 - val_mean_absolute_error: 3.0450 - lr: 5.0000e-04
Epoch 32/1000
2/2 [==============================] - 0s 23ms/step - loss: 22.1937 - mean_squared_error: 22.1937 - rmse: 4.7110 - mean_absolute_error: 3.8927 - val_loss: 14.7190 - val_mean_squared_error: 14.7190 - val_rmse: 3.8365 - val_mean_absolute_error: 3.0694 - lr: 5.0000e-04
Epoch 33/1000
2/2 [==============================] - 0s 27ms/step - loss: 21.9613 - mean_squared_error: 21.9613 - rmse: 4.6863 - mean_absolute_error: 3.6818 

2/2 [==============================] - 0s 22ms/step - loss: 17.9549 - mean_squared_error: 17.9549 - rmse: 4.2373 - mean_absolute_error: 3.3535 - val_loss: 17.2982 - val_mean_squared_error: 17.2982 - val_rmse: 4.1591 - val_mean_absolute_error: 3.2935 - lr: 2.5000e-04
Epoch 59/1000
2/2 [==============================] - 0s 21ms/step - loss: 22.9361 - mean_squared_error: 22.9361 - rmse: 4.7892 - mean_absolute_error: 3.8457 - val_loss: 16.9016 - val_mean_squared_error: 16.9016 - val_rmse: 4.1112 - val_mean_absolute_error: 3.2710 - lr: 2.5000e-04
Epoch 60/1000
2/2 [==============================] - 0s 30ms/step - loss: 22.4445 - mean_squared_error: 22.4445 - rmse: 4.7376 - mean_absolute_error: 3.7725 - val_loss: 16.2646 - val_mean_squared_error: 16.2646 - val_rmse: 4.0329 - val_mean_absolute_error: 3.2306 - lr: 2.5000e-04
Epoch 61/1000
2/2 [==============================] - 0s 33ms/step - loss: 21.1780 - mean_squared_error: 21.1780 - rmse: 4.6020 - mean_absolute_error: 3.5115 - val_loss: 15

2/2 [==============================] - 0s 26ms/step - loss: 20.9280 - mean_squared_error: 20.9280 - rmse: 4.5747 - mean_absolute_error: 3.5733 - val_loss: 15.2071 - val_mean_squared_error: 15.2071 - val_rmse: 3.8996 - val_mean_absolute_error: 3.1471 - lr: 1.2500e-04
Epoch 87/1000
2/2 [==============================] - 0s 29ms/step - loss: 19.4140 - mean_squared_error: 19.4140 - rmse: 4.4061 - mean_absolute_error: 3.4559 - val_loss: 15.3125 - val_mean_squared_error: 15.3125 - val_rmse: 3.9131 - val_mean_absolute_error: 3.1564 - lr: 1.2500e-04
Epoch 88/1000
1/2 [==============>...............] - ETA: 0s - loss: 19.7837 - mean_squared_error: 19.7837 - rmse: 4.4479 - mean_absolute_error: 3.5616
Epoch 00088: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
2/2 [==============================] - 0s 23ms/step - loss: 21.3408 - mean_squared_error: 21.3408 - rmse: 4.6196 - mean_absolute_error: 3.7006 - val_loss: 15.5200 - val_mean_squared_error: 15.5200 - val_rmse: 3.9395 - val

In [11]:
# We want to try different training sample sizes: s
# (1453, 1000, 700, 350, 175)